In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp
from numpy import average, std

In [ ]:
solver_scip = pywraplp.Solver.CreateSolver("SCIP") # Example, creating a SCIP solver
solver_cbc = pywraplp.Solver.CreateSolver("CBC") # Example, creating a CBC solver
solver_glop = pywraplp.Solver.CreateSolver("GLOP") # Example, creating a SCIP solver

In [ ]:
solver = solver_glop

variables = {}

for i in range(I):
  for j in range(J):
    for k in range(K):
      for l in range(L):
        v = solver.NumVar(0, solver.infinity(), "Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " to cover the neighbourhood " + index2neighbourhood(k) + " in shift " + index2shift(i))
        variables[(i,j,k,l)] = v

In [ ]:
objective = solver.Objective()
objective.SetMinimization()
for i in range(number_origins):
  for j in range(number_destinations):
    v = variables[(i,j)]
    cost = transportation_cost_per_unit[(i,j)]
    objective.SetCoefficient(v, cost)

In [ ]:
station_capacity = []

for l in range(L):
  capacity = capacity_at_station[l]
  for i in range(I):
    c = solver.Constraint(-solver.infinity(), capacity)
    for j in range(J):
      for k in range(K):
        v = X[(i,j,k,l)]
        c.SetCoefficient(v, 1)
    station_capacity.append(c)

In [ ]:
nine_minutes = []

for k in range(K):
  for i in range(I):
    c = solver.Constraint(1, solver.infinity())
    for l in range(L):
      is_inside = station_inside[(k,l)]
      for j in range(J):
        v = X[(i,j,k,l)]
        c.SetCoefficient(v, is_inside)
    nine_minutes.append(c)

In [ ]:
solver = pywraplp.Solver.CreateSolver("GLOP")

variables = {}

for i in range(number_origins):
  for j in range(number_destinations):
    v = solver.NumVar(0, solver.infinity(), "Units transported between: " + str((i,j)))
    variables[(i,j)] = v

objective = solver.Objective()
objective.SetMinimization()
for i in range(number_origins):
  for j in range(number_destinations):
    v = variables[(i,j)]
    cost = transportation_cost_per_unit[(i,j)]
    objective.SetCoefficient(v, cost)

stock_constraints = []

for i in range(number_origins):
  stock = stock_at_origin[i]
  c = solver.Constraint(-solver.infinity(), stock)
  for j in range(number_destinations):
    v = variables[(i,j)]
    c.SetCoefficient(v, 1)
  stock_constraints.append(c)

demand_constraints = []

for j in range(number_destinations):
  min_demand = demand_at_destination[j]
  c = solver.Constraint(min_demand, solver.infinity())
  for i in range(number_origins):
    v = variables[(i,j)]
    c.SetCoefficient(v,1)
  demand_constraints.append(c)

result = solver.Solve()

if result == solver.ABNORMAL :
  print("Execution finished by an error")
elif result == solver.FEASIBLE :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(number_origins):
    for j in range(number_destinations):
      v = variables[(i,j)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE :
  print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED :
  print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(number_origins):
    for j in range(number_destinations):
      v = variables[(i,j)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED :
  print("The solution is unbounded")
else :
  print("Unknown error code")